# 네이버 영화 리뷰 감성 분류

In [1]:
!pip install konlpy soynlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 4.5 MB/s 
     |████████████████████████████████| 416 kB 23.4 MB/s 
     |████████████████████████████████| 465 kB 54.5 MB/s 


In [2]:
import konlpy
import pandas as pd
import numpy as np

In [3]:
!wget https://github.com/rickiepark/python-machine-learning-book-3rd-edition/raw/master/ch08/ratings_train.txt -O ratings_train.txt

--2022-11-15 07:35:38--  https://github.com/rickiepark/python-machine-learning-book-3rd-edition/raw/master/ch08/ratings_train.txt
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rickiepark/python-machine-learning-book-3rd-edition/master/ch08/ratings_train.txt [following]
--2022-11-15 07:35:38--  https://raw.githubusercontent.com/rickiepark/python-machine-learning-book-3rd-edition/master/ch08/ratings_train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: ‘ratings_train.txt’

ratings_train.txt   100%[===================>]  13.95M  --.-KB/s    in 0.09s 

`ratings_train.txt` 파일은 하나의 리뷰가 한 행을 구성하며 각 필드는 탭으로 구분되어 있기 때문에 판다스의 `read_csv()` 함수로 간편하게 읽어 들일 수 있다.  
`read_csv()` 는 기본적으로 콤마를 기준으로 필드를 구분하므로 `delimiter='\t'` 으로 지정하여 탭으로 변경한다. 기본적으로 판다스는 빈 문자열을 NaN으로 인식한다. 빈 문자열을 그대로 유지하기 위해 `keep_default_na` 매개변수를 `False`로 지정한다. 

In [4]:
df_train = pd.read_csv('ratings_train.txt', 
                       delimiter='\t', keep_default_na=False)

In [5]:
df_train.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


데이터프레임의 열을 선택하여 `Series` 객체의 `values` 속성을 사용하면 document 열과 label 열을 넘파이 배열로 저장할 수 있다. 각각 훈련 데이터셋의 특성과 타깃 값으로 저장한다. 

In [6]:
X_train = df_train['document'].values
y_train = df_train['label'].values

In [7]:
!wget https://github.com/rickiepark/python-machine-learning-book-3rd-edition/raw/master/ch08/ratings_test.txt -O ratings_test.txt

--2022-11-15 07:35:39--  https://github.com/rickiepark/python-machine-learning-book-3rd-edition/raw/master/ch08/ratings_test.txt
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rickiepark/python-machine-learning-book-3rd-edition/master/ch08/ratings_test.txt [following]
--2022-11-15 07:35:41--  https://raw.githubusercontent.com/rickiepark/python-machine-learning-book-3rd-edition/master/ch08/ratings_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4893335 (4.7M) [text/plain]
Saving to: ‘ratings_test.txt’

ratings_test.txt    100%[===================>]   4.67M  --.-KB/s    in 0.06s   



`ratings_test.txt` 파일에 대해서도 동일한 작업을 수행한다.

In [8]:
df_test = pd.read_csv('ratings_test.txt', 
                      delimiter='\t', keep_default_na=False)

X_test = df_test['document'].values
y_test = df_test['label'].values

In [9]:
print(len(X_train), np.bincount(y_train))

150000 [75173 74827]


In [10]:
print(len(X_test), np.bincount(y_test))

50000 [24827 25173]


훈련 데이터셋과 테스트 데이터셋의 크기를 확인해보면 각각 150000개와 50000개의 샘플을 가지고 있고 양성 클래스와 음성 클래스의 비율은 거의 50%에 가깝다.  

형태소 분석기를 사용해 본격적인 감성 분류 작업을 시작해보자.  

konlypy 는 5개의 한국어 형태소 분석기를 파이썬 클래스로 감싸서 제공하는 래퍼 패키지이다.  
이 예에서는 스칼라로 개발된 open-korean-text 한국어 처리기 Okt 클래스를 사용해본다. 



In [11]:
from konlpy.tag import Okt

okt = Okt()
print(X_train[4])
print(okt.morphs(X_train[4]))

사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
['사이', '몬페', '그', '의', '익살스런', '연기', '가', '돋보였던', '영화', '!', '스파이더맨', '에서', '늙어', '보이기만', '했던', '커스틴', '던스트', '가', '너무나도', '이뻐', '보였다']


한글 문장에서 조사와 어미가 잘 구분되어 출력된 것을 볼 수 있다. '사이몬페그'와 같은 고유 명사는 처리하는데 어려움을 겪고 있다. 완벽하지는 않지만 이 클래스를 사용해 분류 문제를 풀어보자.  

`TfidfVectorzier`는 기본적으로 공백을 기준으로 토큰을 구분하지만 `tokenizer` 매개변수에 토큰화를 위한 사용자 정의 함수를 전달할 수 있다.  
따라서 `okt.morphs` 메서드를 전달하면 형태소 분석을 통해 토큰화를 수행할 수 있다.  
`tokenizer` 매개변수를 사용할 때 패턴 `token_pattern=None`으로 지정하여 `token_pattern` 매개변수가 사용되지 않는다는 경고 메시지가 나오지 않게 한다.  

`TfidfVectorzier`을 `ngram_range=(1,2)`로 설정하여 유니그램과 바이그램을 사용하고 `min_df=3`으로 지정하여 3회 미만으로 등장하는 토큰은 무시한다.  
또한 `max_df=0.9`로 두어 가장 많이 등장하는 상위 10%의 토큰도 무시한다. 이런 작업이 불용어로 생각할 수 있는 토큰을 제거할 것이다.  



In [12]:
import os
from scipy.sparse import save_npz, load_npz
from sklearn.feature_extraction.text import TfidfVectorizer

if not os.path.isfile('okt_train.npz'):
    tfidf = TfidfVectorizer(ngram_range=(1, 2), 
                            min_df=3,
                            max_df=0.9,
                            tokenizer=okt.morphs, 
                            token_pattern=None)
    tfidf.fit(X_train)
    X_train_okt = tfidf.transform(X_train)
    X_test_okt = tfidf.transform(X_test)
    save_npz('okt_train.npz', X_train_okt)
    save_npz('okt_test.npz', X_test_okt)
else:
    X_train_okt = load_npz('okt_train.npz')
    X_test_okt = load_npz('okt_test.npz')

`X_train_okt`와 `X_test_okt`가 준비되었으므로 `SGDClassifier` 클래스를 사용해 감성 분류 문제를 풀어보자.  
탐색할 `SGDClassifier`의 매개변수는 규제를 위한 `alpha` 매개변수이다. `RandomizedSearchCV` 클래스를 사용하기 위해 `loguniform` 함수로 탐색 범위를 지정한다. 여기에서는 `SGDClassifier`의 손실 함수로 로지스틱 손실(`'log'`)을 사용하지만 다른 손실 함수를 매개변수 탐색에 포함할 수 있다. 

In [13]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.utils.fixes import loguniform

sgd = SGDClassifier(loss='log', random_state=1)
param_dist = {'alpha': loguniform(0.0001, 100.0)}

rsv_okt = RandomizedSearchCV(estimator=sgd,
                             param_distributions=param_dist,
                             n_iter=50,
                             random_state=1,
                             verbose=1,
                             n_jobs=-1)
rsv_okt.fit(X_train_okt, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


RandomizedSearchCV(estimator=SGDClassifier(loss='log', random_state=1),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f600374af90>},
                   random_state=1, verbose=1)

In [14]:
print(rsv_okt.best_score_)
print(rsv_okt.best_params_)

0.8251533333333334
{'alpha': 0.0001001581395585897}


테스트 데이터셋 `X_test_okt` 에서 점수도 확인해보자.

In [15]:
rsv_okt.score(X_test_okt, y_test)

0.8189

약 82%의 정확도를 냈습니다. 간단한 작업으로 꽤 좋은 성능을 냈습니다. `konlpy`의 다른 형태소 분석 클래스를 사용하거나 `SGDClassifier` 외에 다른 분류기를 시도하지 않을 이유는 없습니다. 충분한 컴퓨팅 파워가 없다면 사이킷런 0.24버전에서 추가되는 `HalvingRandomSearchCV` 클래스를 사용해 볼 수도 있습니다.

이번에는 또 다른 파이썬 형태소 분석기인 `soynlp`를 사용해 보겠습니다. `soynlp`는 순수하게 파이썬으로 구현된 형태소 분석 패키지입니다.  
`soynlp`는 3개의 토큰화 클래스를 제공합니다. 기본적으로 띄어쓰기가 잘 되어 있다면 `LTokenizer`가 잘 맞습니다. 이외에는 `MaxScoreTokenizer`와 `RegexTokenizer`가 있습니다. 이 예에서는 `LTokenizer`를 사용해 보겠습니다. 먼저 `soynlp.tokenizer`에서 `LTokenizer`를 임포트합니다.

In [16]:
from soynlp.tokenizer import LTokenizer

`LTokenizer` 클래스의 객체를 만든다음 앞에서와 같이 훈련 데이터셋에 있는 샘플(`X_train[4]`) 하나의 형태소를 분석해 보겠습니다.

In [17]:
lto = LTokenizer()

print(lto.tokenize(X_train[4]))

['사이몬페그의', '익살스런', '연기가', '돋보였던', '영화!스파이더맨에서', '늙어보이기만', '했던', '커스틴', '던스트가', '너무나도', '이뻐보였다']


`soynlp`는 말뭉치의 통계 데이터를 기반으로 동작하기 때문에 기본 `LTokenizer` 객체로는 공백으로만 토큰화를 수행합니다. `LTokenizer`에 필요한 통계 데이터를 생성하기 위해서 `WordExtractor`를 사용해 보겠습니다.

In [18]:
from soynlp.word import WordExtractor

`WordExtractor` 객체를 만든 후 `train()` 메서드에 `X_train`을 전달하여 훈련합니다. 훈련이 끝나면 `word_scores()` 메서드에서 단어의 점수를 얻을 수 있습니다.  
반환된 `scores` 객체는 단어마다 결합 점수(cohesion score)와 브랜칭 엔트로피(branching entropy)를 가진 딕셔너리입니다.

In [19]:
word_ext = WordExtractor()
word_ext.train(X_train)
scores = word_ext.word_scores()

training was done. used memory 1.583 Gb
all cohesion probabilities was computed. # words = 85683
all branching entropies was computed # words = 101540
all accessor variety was computed # words = 101540


In [20]:
import math

score_dict = {key: scores[key].cohesion_forward *
              math.exp(scores[key].right_branching_entropy) 
              for key in scores}

이제 이 점수를 `LTokenizer`의 `scores` 매개변수로 전달하여 객체를 만들고 앞에서 테스트한 샘플에 다시 적용해 보겠습니다.

In [21]:
lto = LTokenizer(scores=score_dict)

In [22]:
print(lto.tokenize(X_train[4]))

['사이', '몬페그의', '익살스', '런', '연기', '가', '돋보', '였던', '영화', '!스파이더맨에서', '늙어', '보이기만', '했던', '커스틴', '던스트가', '너무', '나도', '이뻐', '보였다']


단어 점수를 활용했기 때문에 토큰 추출이 훨씬 잘 된 것을 볼 수 있습니다.  
`lto.tokenizer` 메서드를 `TfidVectorizer` 클래스에 전달하여 `konlpy`를 사용했을 때와 같은 조건으로 훈련 데이터셋과 테스트 데이터셋을 변환해 보겠습니다.

In [23]:
if not os.path.isfile('soy_train.npz'):
    tfidf = TfidfVectorizer(ngram_range=(1, 2),
                            min_df=3,
                            max_df=0.9,
                            tokenizer=lto.tokenize, 
                            token_pattern=None)
    tfidf.fit(X_train)
    X_train_soy = tfidf.transform(X_train)
    X_test_soy = tfidf.transform(X_test)
    save_npz('soy_train.npz', X_train_soy)
    save_npz('soy_test.npz', X_test_soy)
else:
    X_train_soy = load_npz('soy_train.npz')
    X_test_soy = load_npz('soy_test.npz')

동일한 `SGDClassifier` 객체와 매개변수 분포를 지정하고 하이퍼파라미터 탐색을 수행해 보겠습니다.

In [24]:
rsv_soy = RandomizedSearchCV(estimator=sgd,
                             param_distributions=param_dist,
                             n_iter=50,
                             random_state=1,
                             verbose=1,
                             n_jobs=-1)
rsv_soy.fit(X_train_soy, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


RandomizedSearchCV(estimator=SGDClassifier(loss='log', random_state=1),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f600374af90>},
                   random_state=1, verbose=1)

`soynlp`를 사용했을 때 최상의 점수와 매개변수는 다음과 같습니다.

In [25]:
print(rsv_soy.best_score_)
print(rsv_soy.best_params_)

0.8141066666666665
{'alpha': 0.0001001581395585897}


In [26]:
rsv_soy.score(X_test_soy, y_test)

0.8085

`Okt`를 사용했을 때 보다는 조금 더 낮지마 약 81% 이상의 정확도를 얻었다.